# Package

In [2]:
import pandas as pd

# importation des données

In [5]:
df_stationary_train = pd.read_csv("df_stationary_train.csv", index_col='date')
df_stationary_train.head()

,UNRATE,TB3MS,RPI,INDPRO,DPCERA3M086SBEA,S&P 500,BUSLOANS,CPIAUCSL,OILPRICEx,M2SL,USREC
date,,,,,,,,,,,
1960-01-31,-0.8,0.30,0.020977,0.091980,0.001204,0.017909,0.011578,-0.006156,0.0,0.001323,0
1960-02-29,-1.1,-0.19,0.014565,0.076964,0.006009,-0.025663,0.011905,-0.003767,0.0,0.002007,0
1960-03-31,-0.2,-1.18,0.006250,0.007961,0.021240,-0.070857,-0.008356,-0.005455,0.0,0.001324,0
1960-04-30,0.0,-1.12,0.006489,-0.025915,0.033752,-0.040442,-0.009098,0.005090,0.0,0.000634,1
1960-05-31,0.0,-0.67,0.007747,-0.018121,0.009040,-0.010090,-0.000359,0.003383,0.0,0.003977,1


# Test de cointégration de Johansen

In [9]:
import pandas as pd
from statsmodels.tsa.vector_ar.vecm import coint_johansen

# Sélection des variables
data_johansen = df_stationary_train[['UNRATE','TB3MS','RPI','INDPRO','DPCERA3M086SBEA',
                                     'S&P 500','BUSLOANS','CPIAUCSL','OILPRICEx','M2SL','USREC']]

# Test de Johansen
jres = coint_johansen(data_johansen, det_order=0, k_ar_diff=2)

# ---- DataFrame TRACE ----
trace_df = pd.DataFrame({
    "Trace Statistic": jres.lr1,
    "Crit. Value 95%": jres.cvt[:, 1],
})
trace_df["Conclusion (Trace)"] = trace_df["Trace Statistic"] > trace_df["Crit. Value 95%"]
trace_df["Conclusion (Trace)"] = trace_df["Conclusion (Trace)"].map({True: "Reject H0", False: "Do not reject H0"})
trace_df.index = [f"r ≤ {i}" for i in range(len(trace_df))]

# ---- DataFrame MAX-EIGEN ----
maxeig_df = pd.DataFrame({
    "Max-Eigen Statistic": jres.lr2,
    "Crit. Value 95%": jres.cvm[:, 1],
})
maxeig_df["Conclusion (Max-Eigen)"] = maxeig_df["Max-Eigen Statistic"] > maxeig_df["Crit. Value 95%"]
maxeig_df["Conclusion (Max-Eigen)"] = maxeig_df["Conclusion (Max-Eigen)"].map({True: "Reject H0", False: "Do not reject H0"})
maxeig_df.index = [f"r ≤ {i}" for i in range(len(maxeig_df))]

# ---- Résumé final du rang r ----
rank_trace = trace_df["Conclusion (Trace)"].eq("Reject H0").sum()
rank_maxeig = maxeig_df["Conclusion (Max-Eigen)"].eq("Reject H0").sum()
rank_final = min(rank_trace, rank_maxeig)

print("\n=== TABLEAU JOHANSEN : TRACE ===")
display(trace_df)

print("\n=== TABLEAU JOHANSEN : MAX-EIGEN ===")
display(maxeig_df)

print("\n=== CONCLUSION FINALE ===")
print(f"Le test de Johansen indique un rang de cointégration r = {rank_final}.")


=== TABLEAU JOHANSEN : TRACE ===


,Trace Statistic,Crit. Value 95%,Conclusion (Trace)
r ≤ 0,1495.692495,285.1402,Reject H0
r ≤ 1,1197.999367,239.2468,Reject H0
r ≤ 2,982.706281,197.3772,Reject H0
r ≤ 3,793.311123,159.5290,Reject H0
r ≤ 4,624.383619,125.6185,Reject H0
r ≤ 5,469.245622,95.7542,Reject H0
r ≤ 6,335.443288,69.8189,Reject H0
r ≤ 7,227.902613,47.8545,Reject H0
r ≤ 8,136.597291,29.7961,Reject H0
r ≤ 9,57.024603,15.4943,Reject H0



=== TABLEAU JOHANSEN : MAX-EIGEN ===


,Max-Eigen Statistic,Crit. Value 95%,Conclusion (Max-Eigen)
r ≤ 0,297.693129,70.5392,Reject H0
r ≤ 1,215.293085,64.5040,Reject H0
r ≤ 2,189.395158,58.4332,Reject H0
r ≤ 3,168.927504,52.3622,Reject H0
r ≤ 4,155.137997,46.2299,Reject H0
r ≤ 5,133.802334,40.0763,Reject H0
r ≤ 6,107.540675,33.8777,Reject H0
r ≤ 7,91.305322,27.5858,Reject H0
r ≤ 8,79.572688,21.1314,Reject H0
r ≤ 9,42.444433,14.2639,Reject H0



=== CONCLUSION FINALE ===
Le test de Johansen indique un rang de cointégration r = 11.


L'ensemble des différentes variables sont donc reliées entre elles à long terme. Dans ce cas, un modèle VECM à estimer.